<style>
    .center-text{
        text-align: center;
    }
</style>

<div class='center-text'>

# Tarea 1 Aprendizaje Automático

### Julian Ricardo Salazar duarte

#### Universidad Nacional de Colombia

#### 2024-1
</div>


- [1. Preprocesamiento de datos](#s1)
- [2. Análisis exploratorio de datos](#s2)


In [1]:
import pandas as pd

<a id="s1"></a>
### 1. Preprocesamiento de datos

In [2]:
def load_data(filename, skiprows=0, sheet_name=None):
    if sheet_name is None:
        data = pd.read_excel(filename, skiprows=skiprows)
    else:
        data = pd.read_excel(filename, skiprows=skiprows, sheet_name=sheet_name)
    return data

In [3]:
def clean_df(df):

    df = df.dropna(how='all')

    for i in range (len(df.columns)):
        if 'Unnamed' in df.columns[i]:
            df = df.rename(columns={df.columns[i]: df.columns[i-1].split()[0] + ' ' + df.iloc[0, i]})
        elif not pd.isna(df.iloc[0, i]):
            df = df.rename(columns={df.columns[i]: df.columns[i] + ' ' + df.iloc[0, i]})

    df = df.drop(df.index[0])
    df = df.dropna(how='any')
    df.columns = df.columns.str.strip()
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.capitalize()
    df['Fecha'] = df['Fecha'].apply(str)
    
    return df

In [4]:
df = load_data('CC FT 17 Formato de Control de Calidad Café de Trillado.xlsx',
               5, ['CONTROL CALIDAD CAFE TRILLADO J', 'Sheet2'])

df1 = df['CONTROL CALIDAD CAFE TRILLADO J']
df2 = df['Sheet2']

df1 = clean_df(df1)
df2 = clean_df(df2)

In [5]:
df_trillado = pd.concat([df1, df2], axis=0)

for col in df_trillado.columns:
    if isinstance(df_trillado[col].iloc[0], str):
        df_trillado[col] = df_trillado[col].str.lower().str.strip()
    if len(set(df_trillado[col])) == 1:
        df_trillado = df_trillado.drop(columns=[col])

df_trillado['Lote'] = df_trillado['Lote'].str.replace('-', '', regex=False).str.lstrip('0')

df_trillado = df_trillado.reset_index(drop=True)

print(df_trillado.shape)

(106, 9)


In [6]:
df_trillado.head()

,Fecha,Lote,Denominación/ marca,Cantidad,%h %,Mallas #,Notas de catación,Puntaje n°,Responsable
0,19-07-22,1190722,madre laura,765.00,10.9,14,"chocolate negro, toque frutal, cuerpo medio, a...",84,lfq
1,19-7-22,9190722,tabi natural,204.00,10.2,14,"frutas maduras, nibs de cacao, acidez brillant...",85,lfq
2,19-07-22,10190722,don mario,165.00,10.7,14,"panela, durazno, miel, acidez brillante citric...",84.5,lfq
3,27-07-22,7190722,don felix,0.45,10.5,14,"moras maduras, chocolate negro, acidez media c...",84.5,lfq
4,31-10-22,1291022,madre laura,105.00,10.7,14,"chocolate negro, toque frutal, cuerpo medio, a...",84,lfq


In [7]:
df_tostion = load_data('CC FT 18 Formato de Tostión.xlsx',
               5, ['TOSTIÓN JERICÓ L'])

df_tostion = df_tostion['TOSTIÓN JERICÓ L']

df_tostion = df_tostion.dropna(how='all', axis=1)
df_tostion = df_tostion.drop(columns=['Tostador'])
df_tostion.columns = df_tostion.columns.str.strip()
df_tostion['Fecha'] = df_tostion['Fecha'].apply(str)

for col in df_tostion.columns:
    if isinstance(df_tostion[col].iloc[0], str):
        df_tostion[col] = df_tostion[col].str.lower().str.strip()
    if len(set(df_tostion[col])) == 1:
        df_tostion = df_tostion.drop(columns=[col])

df_tostion['Lote'] = df_tostion['Lote'].str.replace('-', '', regex=False).str.lstrip('0')

df_tostion = df_tostion.reset_index(drop=True)

print(df_tostion.shape)

(499, 12)


In [8]:
df_tostion.head()

,Fecha,Lote,Origen,Variedad,Proceso,Beneficio,Peso en Verde,Merma,Peso en Tostado,Perfil,Temp. De inicio y final,Tiempo de tueste
0,22-07-22,1190722,jerico,dos mil,tradicional,lavado,9.00,15.000000,7.65,filtrado,175°/191°,08:01:00
1,25-07-22,1190722,jerico,dos mil,tradicional,lavado,3.00,16.666667,2.50,espressso,175°/195°,08:42:00
2,25-07-22,1190722,jerico,dos mil,tradicional,lavado,9.00,16.444444,7.52,filtrado,175°/190°,07:58:00
3,28-07-22,1190722,jerico,dos mil,tradicional,lavado,11.70,16.324786,9.79,filtrado,175°/191°,08:02:00
4,28-07-22,9190722,ciudad bolivar,tabi,natural,natural,0.45,24.444444,0.34,filtrado,150°/186°,08:10:00


In [9]:
print(len(set(df_trillado['Lote']) & set(df_tostion['Lote'])))
print(len(set(df_trillado['Lote'])))
print(len(set(df_tostion['Lote'])))

49
102
55


In [10]:
df = pd.merge(df_trillado, df_tostion, on='Lote', how='inner')
print(len(set(df['Lote'])))
print(df.shape)

49
(526, 20)


In [11]:
df.head()

,Fecha_x,Lote,Denominación/ marca,Cantidad,%h %,Mallas #,Notas de catación,Puntaje n°,Responsable,Fecha_y,Origen,Variedad,Proceso,Beneficio,Peso en Verde,Merma,Peso en Tostado,Perfil,Temp. De inicio y final,Tiempo de tueste
0,19-07-22,1190722,madre laura,765.0,10.9,14,"chocolate negro, toque frutal, cuerpo medio, a...",84,lfq,22-07-22,jerico,dos mil,tradicional,lavado,9.0,15.000000,7.65,filtrado,175°/191°,08:01:00
1,19-07-22,1190722,madre laura,765.0,10.9,14,"chocolate negro, toque frutal, cuerpo medio, a...",84,lfq,25-07-22,jerico,dos mil,tradicional,lavado,3.0,16.666667,2.50,espressso,175°/195°,08:42:00
2,19-07-22,1190722,madre laura,765.0,10.9,14,"chocolate negro, toque frutal, cuerpo medio, a...",84,lfq,25-07-22,jerico,dos mil,tradicional,lavado,9.0,16.444444,7.52,filtrado,175°/190°,07:58:00
3,19-07-22,1190722,madre laura,765.0,10.9,14,"chocolate negro, toque frutal, cuerpo medio, a...",84,lfq,28-07-22,jerico,dos mil,tradicional,lavado,11.7,16.324786,9.79,filtrado,175°/191°,08:02:00
4,19-07-22,1190722,madre laura,765.0,10.9,14,"chocolate negro, toque frutal, cuerpo medio, a...",84,lfq,31-07-22,jerico,dos mil,tradicional,lavado,29.0,15.344828,24.55,filtrado,175°/192°,08:02:00


<a id="s2"></a>
### 2. Análisis exploratorio de datos